In [2]:
import pymongo
import datetime
import csv
import dateutil
import pandas as pd
import random
from faker import Faker as fake
from pandas import DataFrame

# Proyecto

## Paso 1: Crear BBDD de MongoDB
 
* El objetivo de este proyecto es implementar una BBDD similar a la de IMDB MONGODB y python.

* Diseñar una BBDD en MONGO con todas dos colecciones (professional, movie). 

* Crear una colección Professional con los siguientes campos:
	- **name**: string
	- **age**: number
	- **weight**: number
	- **height**: number
	- **isRetired**: boolean
	- **nationality**: string
	- **oscarsNumber**: number
	- **profession**: string
* Crear una colección movie con los siguientes campos:
	- **title**: string
	- **releaseYear**: number
	- **actors**: string[]
	- **nacionality**: string
	- **director**: string[]
	- **writer**: string[]
	- **language**: string
	- **plataforma**: string
	- **isMCU**: boolean
	- **mainCharacterName**: string
	- **producer**: string
	- **distributor**: string
	- **genre**: string

* Cada película tendrá mas de un actor que consistirá en un conjunto de nombres de actor.* 
* Cada película tendrá mas de un guionista que consistirá en un conjunto de nombres de guionistas.
* Cada película tendrá mas de un director que consistirá en un conjunto de nombres de direct


## Paso 2: Implementación de la BBDD
* Implementar utilizando MONGO Compass y PyMongo la base de datos diseñada en el apartado anterior.

* Añadiendo los suficientes documentos en cada colección para realizar todas las combinaciones posibles de todos los campos.

In [4]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client.IMDB
fake = fake(['es_ES','de_DE','es_AR','it_IT','en_UK','fr_FR','en_US','fi_FI'])

In [157]:
professions = ["fontanero", "programador", "atleta", "musico", "cartero", "saltador", "cajero", "matematico", "doctor", "cientifico", "electricista", "tejedor", "militar", "zapatero", "empresario", "politico", "profesor", "psicologo"]
nationality = ["frances", "colombiano", "estounidense", "español", "suizo", "austriaco", "belga", "mexicano", "finlandes", "uruguayo", "italiano", "canadiense"]
titles = ["El divorcio", "Titanic", "Monstruos SA", "Lo que el viento se llevo", "Maquinistas", "HP 3", "Shrek 3", "Shrek Ultimatum", "Shrek 2", "HP 8", "Matrix", "El resplandor", "El internado", "Oppenheimer", "Barbie", "La sirenita", "La Momia", "Interstellar", "Aventuras misteriosas", "Viajes al mas alla", "Star wars 34", "Star wars: contraataca"]
plataformas = ["HBO", "A3Player", "hulu", "DAZN", "Netflix", "MiTele", "Movistar+", "Amazon", "Youtube"]
generos = ["Terror", "Comedia", "Romantica", "Historica", "Ciencia Ficcion", "Suspense", "Documental"]
oscars = [0,0,0,0,0,0,0,1,2,3,4,5,1,1,2,7,5,4,1,2,0,0,8]

In [55]:
collection = db.professionals

ls = [{"name": fake.name(),"age": random.randint(0, 90),"weight": random.randint(30, 150),"height": random.randint(100, 210),"isRetired": False,"nationality": random.choice(nationality),"oscarsNumber": random.choice(oscars),"profession": random.choice(professions)}
 for _ in range(10)]

ids = collection.insert_many(ls).inserted_ids
print(ids)

[ObjectId('64d0a4f2505c46efac161f5e'), ObjectId('64d0a4f2505c46efac161f5f'), ObjectId('64d0a4f2505c46efac161f60'), ObjectId('64d0a4f2505c46efac161f61'), ObjectId('64d0a4f2505c46efac161f62'), ObjectId('64d0a4f2505c46efac161f63'), ObjectId('64d0a4f2505c46efac161f64'), ObjectId('64d0a4f2505c46efac161f65'), ObjectId('64d0a4f2505c46efac161f66'), ObjectId('64d0a4f2505c46efac161f67'), ObjectId('64d0a4f2505c46efac161f68')]


In [158]:
collection = db.movies

ms = [{"title": random.choice(titles),"releaseYear": random.randint(1910, 2023),"actors": [fake.name(),fake.name()], "nationality":random.choice(nationality),"directors": [fake.name(),fake.name()],"writers": [fake.name(),fake.name()],"language":random.choice(nationality),"platform":random.choice(plataformas),"isMCU": False,"maincharac": fake.name(),"producer": fake.name(),"distributor": random.choice(plataformas),"genre": random.choice(generos)}
 for _ in range(10)]

ids = collection.insert_many(ms).inserted_ids
print(ids)

[ObjectId('64d0b5d3505c46efac161fa6'), ObjectId('64d0b5d3505c46efac161fa7'), ObjectId('64d0b5d3505c46efac161fa8'), ObjectId('64d0b5d3505c46efac161fa9'), ObjectId('64d0b5d3505c46efac161faa'), ObjectId('64d0b5d3505c46efac161fab'), ObjectId('64d0b5d3505c46efac161fac'), ObjectId('64d0b5d3505c46efac161fad'), ObjectId('64d0b5d3505c46efac161fae'), ObjectId('64d0b5d3505c46efac161faf'), ObjectId('64d0b5d3505c46efac161fb0'), ObjectId('64d0b5d3505c46efac161fb1')]


# Paso 3: Alimentar la Colección professional con CSV
* Crear un fichero en formato CSV con datos de profesionales y añadir dichos datos a la base de datos. 



In [307]:
collection = db.professionals

In [56]:
professionals_csv = pd.read_csv('professionals.csv')

professionals_dict = professionals_csv.to_dict("records")

ids = collection.insert_many(professionals_dict).inserted_ids

# Paso 4: Realizar las siguientes consultas en python

* Mostrar todos los profesionales cuyo nombre empiece por “C”.


In [310]:
cursor = collection.find({"name": {"$regex": "^C"}}, {"_id":0, "name": 1, "profession": 1}).sort("name",1)


for data in cursor:
    print(data)

{'name': 'Cirillo Trussardi', 'profession': 'tejedor'}
{'name': 'Ciro Travaglia', 'profession': 'cartero'}


Mostrar todos los profesionales que midan más de 170 cm y pesen menos de 85Kg.

In [188]:
cursor = collection.aggregate ([{"$match":
                                 {"$and" : [ {"weight" : {"$lt" : 85}}, 
                                           {"height": {"$gt": 170}}]}},
                               {"$project": {"_id": 0,
                                              "Nombre": "$name",
                                              "Altura ": "$weight",
                                              "Peso:" : "$height"}}])

for data in cursor:
    print(data)

{'Nombre': 'Nicole Gray', 'Altura ': 79, 'Peso:': 210}
{'Nombre': 'Viktoria Hübel-Wesack', 'Altura ': 64, 'Peso:': 171}
{'Nombre': 'Victoria Caceres', 'Altura ': 65, 'Peso:': 180}
{'Nombre': 'Sr(a). Martina Leiva', 'Altura ': 61, 'Peso:': 183}
{'Nombre': 'Dominique Hubert', 'Altura ': 48, 'Peso:': 171}


Mostrar solo el nombre, apellidos y numero de oscars de los profesionales que tengan al menos 2 oscars.


In [200]:
cursor = collection.aggregate ([{"$match":
                                 {"oscarsNumber" : {"$gte" : 2}}},
                               {"$project": {"_id": 0,
                                              "Nombre y apellidos": "$name",
                                              "Oscars:" : "$oscarsNumber"}}])

for data in cursor:
    print(data)

{'Nombre y apellidos': 'Antti Aalto', 'Oscars:': 2}
{'Nombre y apellidos': 'Tim Francis', 'Oscars:': 7}
{'Nombre y apellidos': 'Viktoria Hübel-Wesack', 'Oscars:': 4}
{'Nombre y apellidos': 'Victoria Caceres', 'Oscars:': 5}
{'Nombre y apellidos': 'Sr(a). Martina Leiva', 'Oscars:': 7}
{'Nombre y apellidos': 'Dimas Salmerón', 'Oscars:': 2}
{'Nombre y apellidos': 'Elmeri Keränen', 'Oscars:': 5}
{'Nombre y apellidos': 'Dr Kathleen Hughes', 'Oscars:': 7}
{'Nombre y apellidos': 'Luchino Palazzo-Muti', 'Oscars:': 7}
{'Nombre y apellidos': 'Fernando Ferrero Roman', 'Oscars:': 4}


Mostrar el numero de profesionales retirados, el número de profesionales no retirados y el total de profesionales de la colección.

In [240]:
cursor = collection.aggregate([{'$group': {'_id': "$isRetired",
                                            'Total': {'$sum': { '$cond': ["$isRetired", 1, 1]}}}}])

for data in cursor:
    print(data)

{'_id': False, 'Total': 13}
{'_id': True, 'Total': 8}


Mostrar nombre, apellidos, nacionalidad y año de nacimiento de aquellos profesionales que tengan una edad mayor que 20 ordenados de manera ascendente.

In [269]:
cursor = collection.aggregate ([{"$match":
                                 {"age" : {"$gt" : 20}}},
                               {"$project": {"_id": 0,
                                              "Nombre": "$name",
                                              "Nacionalidad" : "$nationality",
                                            "Edad": "$age"}},
                                {"$sort": {"Edad": 1}}])

#Convertimos en dataframe y añadimos columna de año de nacimiento
df = DataFrame(list(cursor))
Año = []
for i in df['Edad']:
    Año.append(2023 - i)

df['Año de nacimiento'] = Año

df

,Nombre,Nacionalidad,Edad,Año de nacimiento
0,Nicole Gray,estounidense,24,1999
1,Robby Vollbrecht,español,24,1999
2,Fernando Ferrero Roman,estounidense,25,1998
3,Diane Brookes,estounidense,28,1995
4,Prof. Arne Eimer,italiano,28,1995
5,Sr(a). Martina Leiva,finlandes,36,1987
6,Luchino Palazzo-Muti,italiano,46,1977
7,Victoria Caceres,suizo,48,1975
8,Griselda Cendron,colombiano,49,1974
9,Cirillo Trussardi,austriaco,56,1967


Hacer lo mismo que en el anterior punto pero como máximo muestre 10 documentos.

In [222]:
cursor = collection.aggregate ([{"$match":
                                 {"age" : {"$gt" : 20}}},
                               {"$project": {"_id": 0,
                                              "Nombre": "$name",
                                              "Nacionalidad" : "$nationality",
                                            "Edad": "$age"}},
                                {"$sort": {"Edad": 1}},
                                {"$limit": 10}])

#Convertimos en dataframe y añadimos columna de año de nacimiento
df = DataFrame(list(cursor))
Año = []
for i in df['Edad']:
    Año.append(2023 - i)

df['Año de nacimiento'] = Año

df

,Nombre,Nacionalidad,Edad,Año de nacimiento
0,Robby Vollbrecht,español,24,1999
1,Nicole Gray,estounidense,24,1999
2,Fernando Ferrero Roman,estounidense,25,1998
3,Prof. Arne Eimer,italiano,28,1995
4,Diane Brookes,estounidense,28,1995
5,Sr(a). Martina Leiva,finlandes,36,1987
6,Luchino Palazzo-Muti,italiano,46,1977
7,Victoria Caceres,suizo,48,1975
8,Griselda Cendron,colombiano,49,1974
9,Cirillo Trussardi,austriaco,56,1967


Mostrar el numero total de profesionales agrupados por profesión.

In [260]:
pipeline = [{"$group": {"_id": "$profession", "total": {'$sum': 1}}}]

cursor = collection.aggregate(pipeline)

for data in cursor:
    print(data)

{'_id': 'militar', 'total': 3}
{'_id': 'cientifico', 'total': 2}
{'_id': 'atleta', 'total': 1}
{'_id': 'cartero', 'total': 3}
{'_id': 'psicologo', 'total': 1}
{'_id': 'tejedor', 'total': 1}
{'_id': 'cajero', 'total': 7}
{'_id': 'fontanero', 'total': 1}
{'_id': 'profesor', 'total': 2}


Filtrar el anterior resultado para que solo aparezcan los profesionales con más de un oscar.

In [261]:
pipeline = [{"$match": {"oscarsNumber" : {"$gt" : 1}}},
            {"$group": {"_id": "$profession", "total": {'$sum': 1}}}]

cursor = collection.aggregate(pipeline)

for data in cursor:
    print(data)

{'_id': 'fontanero', 'total': 1}
{'_id': 'cajero', 'total': 3}
{'_id': 'militar', 'total': 3}
{'_id': 'cartero', 'total': 1}
{'_id': 'cientifico', 'total': 1}
{'_id': 'atleta', 'total': 1}


Filtrar lo anterior para que solo muestre profesionales que hayan nacido entre 1970 y 1990.

In [286]:
pipeline = [{"$match":
             {"$and" : [ {"oscarsNumber" : {"$gt" : 1}},
                       {"age": {"$lt": 53,"$gt":33}}]}},
            {"$group": {"_id": "$profession", "total": {'$sum': 1}}}]

cursor = db.professionals.aggregate(pipeline)

for data in cursor:
    print(data)

{'_id': 'militar', 'total': 1}
{'_id': 'cientifico', 'total': 1}
{'_id': 'atleta', 'total': 1}


Volcar toda información del anterior punto en un Dataframe.

In [275]:
pipeline = [{"$match":
             {"$and" : [ {"oscarsNumber" : {"$gt" : 1}},
                       {"age": {"$lt": 53,"$gt":33}}]}},
            {"$group": {"_id": "$profession", "total": {'$sum': 1}}}]
cursor = db.professionals.aggregate(pipeline)
df = DataFrame(list(cursor))
df

,_id,total
0,militar,1
1,cientifico,1
2,atleta,1


# Paso 5: Alimentar la Colección movie con CSV
* Crear un fichero en formato CSV con datos de pelicylas y añadir dichos datos a la base de datos. 

In [297]:
collection = db.movies

movies_csv = pd.read_csv('movies.csv')

In [298]:
#PREPARAMOS EL DATAFRAME PARA MONGO

#renombrar columnas para futuras funciones

movies_csv = movies_csv.rename(columns={'actors[0]':'actors0','actors[1]':'actors1',
                                        'writers[0]':'writers0','writers[1]':'writers1',
                                       'directors[0]':'directors0','directors[1]':'directors1'})

#creamos columnas agrupadas

movies_csv['actors2'] = movies_csv.actors1.str.cat(movies_csv.actors0,sep=',')
movies_csv['directors2'] = movies_csv.directors1.str.cat(movies_csv.directors0,sep=',')
movies_csv['writers2'] = movies_csv.writers1.str.cat(movies_csv.writers0,sep=',')

#pasamos columnas agrupadas a array

actors = []
for i in movies_csv['actors2']:
    actors.append(i.split(","))

movies_csv['actors'] = actors

writers = []
for i in movies_csv['writers2']:
    writers.append(i.split(","))

movies_csv['writers'] = writers

directors = []
for i in movies_csv['directors2']:
    directors.append(i.split(","))

movies_csv['directors'] = directors

#ordenamos y eliminamos columnas innecesarias

movies_csv.drop(['actors1','actors0','directors1','writers1','directors0','writers0','actors2','directors2','writers2'], axis=1, inplace=True)

movies_csv = movies_csv[['title', 'releaseYear','actors', 'nationality','directors',
       'writers', 'language', 'platform', 'isMCU',
       'maincharac', 'producer', 'distributor', 'genre']]


In [153]:
movies_dict = movies_csv.to_dict("records")

ids = collection.insert_many(movies_dict).inserted_ids

In [5]:
collection = db.movies

# Paso 6: Realizar las siguientes consultas en python
* Mostrar todas las películas que sean del MCU.


In [321]:
cursor = collection.find({"isMCU": True}, {"_id":0, "title": 1, "releaseYear": 1}).sort("title",-1)

for data in cursor:
    print(data)

{'title': 'Oppenheimer', 'releaseYear': 1991}
{'title': 'La sirenita', 'releaseYear': 2005}
{'title': 'Interstellar', 'releaseYear': 1912}
{'title': 'HP 8', 'releaseYear': 2015}
{'title': 'El divorcio', 'releaseYear': 2020}


* Mostrar solo los títulos de películas que tengas al menos 3 actores o 2 directores o 1 guionista.

In [334]:
pipeline = [{"$match" :
               {"$or" : [ {"actors.3" : {"$exists" : True}},
                         {"directors.2": {"$exists": True}},
                         {"writers.1": {"$exists": True}}]}},
           {"$project": {"_id": 0,
                         "Titulo": "$title"}}]

cursor = collection.aggregate (pipeline)

for data in cursor:
    print(data)

{'Titulo': 'El divorcio'}
{'Titulo': 'Monstruos SA'}
{'Titulo': 'Shrek 2'}
{'Titulo': 'La sirenita'}
{'Titulo': 'Shrek Ultimatum'}
{'Titulo': 'Barbie'}
{'Titulo': 'Oppenheimer'}
{'Titulo': 'La sirenita'}
{'Titulo': 'El internado'}
{'Titulo': 'Interstellar'}
{'Titulo': 'Titanic'}
{'Titulo': 'Matrix'}
{'Titulo': 'Maquinistas'}
{'Titulo': 'El divorcio'}
{'Titulo': 'Barbie'}
{'Titulo': 'HP 8'}
{'Titulo': 'Star wars: contraataca'}
{'Titulo': 'Barbie'}


* Mostrar el titulo de la película y el año de estreno ordenado por plataforma. 

In [347]:
pipeline = [{"$project": {"_id": 0,
                          "Nombre": "$title",
                          "Año": "$releaseYear",
                          "Plataforma" : "$platform"}},
            {"$sort": {"Plataforma": 1}}]

cursor = collection.aggregate (pipeline)

for data in cursor:
    print(data)

{'Nombre': 'Barbie', 'Año': 2023, 'Plataforma': 'A3Player'}
{'Nombre': 'El internado', 'Año': 1919, 'Plataforma': 'Amazon'}
{'Nombre': 'Star wars: contraataca', 'Año': 2007, 'Plataforma': 'Amazon'}
{'Nombre': 'El divorcio', 'Año': 2020, 'Plataforma': 'DAZN'}
{'Nombre': 'La sirenita', 'Año': 1998, 'Plataforma': 'DAZN'}
{'Nombre': 'Monstruos SA', 'Año': 2003, 'Plataforma': 'HBO'}
{'Nombre': 'Interstellar', 'Año': 1912, 'Plataforma': 'HBO'}
{'Nombre': 'Barbie', 'Año': 1995, 'Plataforma': 'HBO'}
{'Nombre': 'Oppenheimer', 'Año': 1991, 'Plataforma': 'MiTele'}
{'Nombre': 'Titanic', 'Año': 1939, 'Plataforma': 'MiTele'}
{'Nombre': 'Matrix', 'Año': 1998, 'Plataforma': 'MiTele'}
{'Nombre': 'Maquinistas', 'Año': 2021, 'Plataforma': 'MiTele'}
{'Nombre': 'Shrek 2', 'Año': 2014, 'Plataforma': 'Netflix'}
{'Nombre': 'Shrek Ultimatum', 'Año': 2020, 'Plataforma': 'Youtube'}
{'Nombre': 'Barbie', 'Año': 1911, 'Plataforma': 'Youtube'}
{'Nombre': 'HP 8', 'Año': 2015, 'Plataforma': 'Youtube'}
{'Nombre': 'La s

* Mostrar los datos de todas las 10 ultima películas de un director concreto que en los últimos 10 años ordenadas por de la más antigua a la más nueva.

In [408]:
pipeline = [{"$match" :
             {"directors":{"$eq": "Heidi Rantala-Anttila"}}},
           {"$project": {"_id": 0,
                         "Titulo": "$title",
                        "Año": "$releaseYear",
                        "Director": "$directors"}},
           {"$sort": {"Año": 1}},
           {"$limit": 10}]

cursor = collection.aggregate (pipeline)
for data in cursor:
    print(data)

{'Titulo': 'Shrek Ultimatum', 'Año': 2020, 'Director': ['Heidi Rantala-Anttila', 'Sybilla Lübs']}


* Mostrar el titulo de cada película, el numero de actores, directores y guionistas por película de los 5 últimos años ordenados por titulo de película.


In [422]:
pipeline = [{"$match" : {"releaseYear" : {"$gt" : 2018}}},
           {"$project": {"_id": 0,
                         "Titulo": "$title",
                        "Nº Actores": {"$size" : "$actors"},
                         "Nº Directores": {"$size" : "$directors"},
                         "Nº Escritores": {"$size" : "$writers"}}},
           {"$sort": {"Titulo": 1}}]

cursor = collection.aggregate (pipeline)
for data in cursor:
    print(data)

{'Titulo': 'Barbie', 'Nº Actores': 2, 'Nº Directores': 2, 'Nº Escritores': 2}
{'Titulo': 'El divorcio', 'Nº Actores': 2, 'Nº Directores': 2, 'Nº Escritores': 2}
{'Titulo': 'Maquinistas', 'Nº Actores': 2, 'Nº Directores': 2, 'Nº Escritores': 2}
{'Titulo': 'Shrek Ultimatum', 'Nº Actores': 2, 'Nº Directores': 2, 'Nº Escritores': 2}


* Mostrar cuantos directores hacen películas en cada idioma que tengas almacenado en la colección.

In [439]:
pipeline = [{"$project": {"language": 1, "Nº Directores": {"$size": "$directors"}}},
            {"$group": {"_id": "$language", "Total Directores": {"$sum": "$Nº Directores"}}}]

cursor = collection.aggregate(pipeline)
for data in cursor:
    print(data)

{'_id': 'italiano', 'Total Directores': 6}
{'_id': 'español', 'Total Directores': 10}
{'_id': 'aleman', 'Total Directores': 4}
{'_id': 'ingles', 'Total Directores': 6}
{'_id': 'finlandes', 'Total Directores': 6}
{'_id': 'frances', 'Total Directores': 4}


* Mostrar cuantos actores hacen películas en cada nacionalidad que tengas almacenada en la colección.

In [440]:
pipeline = [{"$project": {"nationality": 1, "Nº Actores": {"$size": "$actors"}}},
            {"$group": {"_id": "$nationality", "Total Actores": {"$sum": "$Nº Actores"}}}]

cursor = collection.aggregate(pipeline)
for data in cursor:
    print(data)

{'_id': 'colombiano', 'Total Actores': 4}
{'_id': 'suizo', 'Total Actores': 2}
{'_id': 'belga', 'Total Actores': 2}
{'_id': 'estounidense', 'Total Actores': 2}
{'_id': 'uruguayo', 'Total Actores': 2}
{'_id': 'italiano', 'Total Actores': 4}
{'_id': 'español', 'Total Actores': 2}
{'_id': 'mexicano', 'Total Actores': 4}
{'_id': 'finlandes', 'Total Actores': 8}
{'_id': 'austriaco', 'Total Actores': 6}


* Mostrar el nombre y apellidos de todos los actores guardados en la colección películas sin repetición.

In [16]:
def personas_unicas(x):
    pipeline = [{"$project": {"_id": 0,"actores": x}}]

    cursor = collection.aggregate(pipeline)    

    lista_actores = []
    for i in cursor:
        for actores in i.values():
            for actor in actores:
                if actor not in lista_actores:
                    lista_actores.append(actor)
    return(lista_actores)
    

personas_unicas("$actors")

['Katrina Gonzalez',
 'Sylvie De Oliveira',
 'Ramona Contrafatto',
 'Belinda Brown',
 'Anaïs Marie du Mathieu',
 'Alwina Mitschke',
 'Allan Virtanen',
 'Hannu Uotila',
 'Prof. Alexandra Meyer MBA.',
 'Hannes Kostolzin MBA.',
 'Lucila Donoso',
 'Bernadette Cordier de la Durand',
 'Stéphane Wagner',
 'Noémi Neveu-Martinez',
 'Dott. Francesca Lippomano',
 'Bernardo Lamas Saura',
 'Mia Ortiz',
 'Fanny Pedraza España',
 'Manon Barthelemy',
 'Abril Agustin Martinez',
 'Segismundo del Carrasco',
 'Lola Dylan Luna',
 'Aurélie Coste',
 'Ubaldo Morricone',
 'Giacinto Liguori',
 'Bienvenida Nélida Bejarano Cárdenas',
 'Kelly Wood-Cook',
 'Carolyn Savage-Stokes',
 'Derek Brown',
 'Valeri Lorch-Täsche',
 'Rodney Bullock',
 'Sr(a). Santino Benitez',
 'Jordana Vilanova',
 'Sr(a). Pilar Maldonado',
 'Noémi Gérard de la Marchand',
 'Robert Laine']

In [ ]:
#manera felix

distinct_actors = movie_collection.distinct("actors")
for actor in distinct_actors:
    print(actor)

* Mostrar el número de actores, directores y guionistas distintos almacenados en vuestra colección películas.

In [22]:
print(f'El numero de actores unicos es: {len(personas_unicas("$actors"))}')
print(f'El numero de directores unicos es: {len(personas_unicas("$directors"))}')
print(f'El numero de guionistas unicos es: {len(personas_unicas("$writers"))}')

El numero de actores unicos es: 36
El numero de directores unicos es: 36
El numero de guionistas unicos es: 36


* Mostrar el titulo de película, año de estreno y director de las 5 películas ordenadas de mas antigua a mas nuevas entre los años 1975 y 2000.

In [30]:
pipeline = [{"$match" :
             {"releaseYear":{"$lt": 2000,"$gt": 1975}}},
           {"$project": {"_id": 0,
                         "Titulo": "$title",
                        "Año": "$releaseYear",
                        "Director": "$directors"}},
           {"$sort": {"Año": 1}},
           {"$limit": 5}]

cursor = collection.aggregate(pipeline)
for data in cursor:
    print(data)

{'Titulo': 'Oppenheimer', 'Año': 1991, 'Director': ['Lasse Hartikainen', 'Kevin Walker-Spencer']}
{'Titulo': 'Barbie', 'Año': 1995, 'Director': ['Kim Kerr-Akhtar', 'Amy Stone']}
{'Titulo': 'La sirenita', 'Año': 1998, 'Director': ['Mr Matthew Brooks', 'Corinne Mallet']}
{'Titulo': 'Matrix', 'Año': 1998, 'Director': ['Johannes Hänninen-Saarinen', 'Valentina Lara Acuña']}


* Volcar toda la información del punto anterior en un Dataframe 

In [29]:
pipeline = [{"$match" :
             {"releaseYear":{"$lt": 2000,"$gt": 1975}}},
           {"$project": {"_id": 0,
                         "Titulo": "$title",
                        "Año": "$releaseYear",
                        "Director": "$directors"}},
           {"$sort": {"Año": 1}},
           {"$limit": 5}]

cursor = collection.aggregate (pipeline)

df = DataFrame(list(cursor))

df

,Titulo,Año,Director
0,Oppenheimer,1991,"[Lasse Hartikainen, Kevin Walker-Spencer]"
1,Barbie,1995,"[Kim Kerr-Akhtar, Amy Stone]"
2,La sirenita,1998,"[Mr Matthew Brooks, Corinne Mallet]"
3,Matrix,1998,"[Johannes Hänninen-Saarinen, Valentina Lara Ac..."


In [35]:
#  rewind - solo con find, no con aggregate
#       movies_df = pd.DataFrame.from_records(cursor.rewind())